# INS

pyquaternion 0.9.2

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division
import cv2         # opencv itself
import numpy as np # matrix manipulations
from matplotlib import pyplot as plt           # this lets you draw inline pictures in the notebooks
import pylab                                   # this allows you to control figure size 
pylab.rcParams['figure.figsize'] = (10.0, 10.0) # this controls figure size in the notebook

In [4]:
from math import sin, cos, atan2, pi, sqrt, asin
from math import radians as deg2rad
from math import degrees as rad2deg

In [3]:
#your python code here
def normalize(x, y, z):
	"""Return a unit vector"""
	norm = sqrt(x * x + y * y + z * z)

	# already a unit vector
	if norm == 1.0:
		return (x, y, z)

	if norm > 0.0:
		inorm = 1/norm
		x *= inorm
		y *= inorm
		z *= inorm
	else:
		raise Exception('division by zero: {} {} {}'.format(x, y, z))
	return (x, y, z)

def skew(a):
	return a


def normalize_q(q):
	"""Return a unit quaternion"""
	norm = sqrt(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2)

	# already a unit quaternion
	if norm == 1.0:
		return q

	if norm > 0:
		inorm = 1/norm
		qo = [x*inorm for x in q]
	else:
		raise Exception('normalize div by 0')
	return qo


def quat2euler(q):
	"""
	Returns the euler representation (in degrees) of a quaternion. Note, the
	heading is wrapped between 0-360 degrees.

	In:
		[q0 q1 q2 q3] = [w x y z]
	out:
		(roll, pitch, yaw) in degrees
	"""
	q0, q1, q2, q3 = q
	roll = atan2(2.0*q2*q3-2.0*q0*q1, 2.0*q0*q0+2.0*q3*q3-1.0)
	pitch = -asin(2.0*q1*q3+2.0*q0*q2)
	heading = atan2(2.0*q1*q2-2.0*q0*q3, 2.0*q0*q0+2.0*q1*q1-1.0)

	heading = heading if heading <= 2*pi else heading-2*pi
	heading = heading if heading >= 0 else heading+2*pi

	return map(rad2deg, (roll, pitch, heading))

In [ ]:
def ecef(lat, lon, H):
	# phi = lat
	# lambda = lon
	e = 1.0
	re = 6378137.0  # m
	rm = re * (1.0 - e**2) / pow(1.0 - e**2 * sin(lat)**2, 3.0 / 2.0)
	rn = re / sqrt(1.0 - e**2 * sin(lat)**2)
	x = (rn + H) * cos(lat) * cos(lon)
	y = (rn + H) * cos(lat) * sin(lon)
	z = (rm + H) * sin(lat)
	return x, y, z


def normalizeQuaternion(w, x, y, z):
	m = sqrt(w**2 + x**2 + y**2 + z**2)
	return w/m, x/m, y/m, z/m

In [6]:
class EOM(object):
	"""
	EoM for navigation
	"""
	def __init__(self):
		self.t = 0.0
		self.rk = RK4(self.eom)
		self.epoch = dt.datetime.now()

	@staticmethod
	def eom(t, X, u):
		"""
		X = [vx vy vz px py pz qw qx qy qz]
		v - velocity
		p - position
		q - quaternion (orientation)

		u = [fx fy fz wx wy wz]
		f - force
		w - angular velocity
		"""
		q = X[6:]
		f = u[0:3]
		wx, wy, wz = u[3:]
		p = X[3:6]
		v = X[0:3]
		wie = np.array([0, 0, 7.292115E-15])
		Ceb = np.eye(3)
		W = np.array([
			(0, wz, -wy, wx),
			(-wz, 0, wz, -wy),
			(wy, -wx, 0, wz),
			(-wx, -wy, -wz, 0)
		])
		vd = -2.0*np.cross(wie, np.cross(wie, v))-np.cross(wie, np.cross(wie, p)) + Ceb.dot(f)
		pd = v
		qd = 0.5 * W.dot(q)

		# print('vd', vd)
		# print('pd', pd)
		# print('qd', qd)

		XX = np.hstack((vd, pd, qd))
		return XX

	def step(self, X, u):
		delta = (dt.datetime.now() - self.epoch).total_seconds()
		# t = self.epoch.total_seconds()
		t = self.t
		# X = self.X
		# dt = self.dt
		X = self.rk.step(X, u, t, delta)

		# fix q
		w, x, y, z = normalizeQuaternion(*X[6:])
		X[6] = w
		X[7] = x
		X[8] = y
		X[9] = z

		self.t += delta
		self.epoch = dt.datetime.now()
		return X